# Build Document Research - Multi-Agent RAG system


**Objective**: a multi-agent RAG tool designed to help you ask questions about your documents and receive fact-checked, avoid hallucination answers by using reflection pattern asking system to evaluate the answer.


## Install Required Libraries


In [1]:
%%capture
%pip install langchain-ibm==0.3.10
%pip install langchain==0.3.23
%pip install langchain_community==0.3.21
%pip install langchain-openai==0.3.27
%pip install langgraph==0.6.6
%pip install pygraphviz==1.14
%pip install docling==2.15.0
%pip install docling-core==2.16.0
%pip install docling-ibm-models==3.3.0
%pip install docling-parse==3.1.2
%pip install langchain-text-splitters==0.3.5
%pip install 'unstructured[ocr]==0.16.17'

In [2]:
%pip install langfuse
%pip install python-dotenv
%pip install pymupdf
%pip install chromadb
%pip install rank_bm25

  Using cached googleapis_common_protos-1.72.0-py3-none-any.whl.metadata (9.4 kB)
Using cached googleapis_common_protos-1.72.0-py3-none-any.whl (297 kB)
  Attempting uninstall: wrapt
    Found existing installation: wrapt 2.0.0
    Uninstalling wrapt-2.0.0:
      Successfully uninstalled wrapt-2.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10/10 [langfuse]  9/10 [langfuse]s-common-protos]
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.5/22.5 MB 15.1 MB/s  0:00:015.0 MB/s eta 0:00:01:01
Note: you may need to restart the kernel to use updated packages.
  Using cached uvicorn-0.38.0-py3-none-any.whl.metadata (6.8 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached google_auth-2.43.0-py2.py3-

In [3]:
## Constants
HYBRID_RETRIEVER_WEIGHTS = [0.4, 0.6] # [ bm25, vector_retriever ]
VECTOR_SEARCH_K = 10
TOP_K = 3

In [ ]:
from dotenv import load_dotenv
import os

# Load env
load_dotenv() 

LANGFUSE_SECRET_KEY = os.getenv("LANGFUSE_SECRET_KEY")
LANGFUSE_PUBLIC_KEY = os.getenv("LANGFUSE_PUBLIC_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGFUSE_BASE_URL = os.getenv("LANGFUSE_BASE_URL")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY 

## Import Required Libraries


In [ ]:
# LangGraph modules
from langgraph.graph import StateGraph, END, START

# Type Safety and State Management
from typing import TypedDict, Annotated, List, Literal
from pydantic import BaseModel, Field
import operator
from pprint import pprint

# Graph visualization
from IPython.display import Image, display

# LangChain and LLM 
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# Initialize the language model - using gpt-4o-mini for cost-effective experimentation
# The Open API was imported outside
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

## Process documents and Retrieve
